In [73]:
import numpy as np
from numpy import savetxt
import pandas as pd
import matplotlib.pyplot as plt

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [17]:
df = pd.read_csv("data/cleaned_steam_data.csv", encoding='utf8', index_col=0)

In [23]:
drop_cols = ['funny', 'is_early_access_review', 'recommendation', 'review',
            'title', 'cleaned_reviews']
df.drop(drop_cols, axis=1, inplace=True)
df.head()

,helpful,hour_played,Year,Month,Day,encoded_1,encoded_2,encoded_3,encoded_4,encoded_5,...,encoded_1335,encoded_1336,encoded_1337,encoded_1338,encoded_1339,encoded_1340,encoded_1341,encoded_1342,encoded_1343,encoded_1344
0,4,578,2019,2,10,0,0,0,0,0,...,523,170,1193,523,264,608,307,1143,794,1336
1,0,184,2019,2,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,811
2,0,892,2019,2,7,0,0,0,0,0,...,1106,44,921,633,776,902,794,239,824,997
3,1086,676,2018,6,14,0,0,0,0,0,...,878,178,394,1073,1258,1395,923,932,1015,285
4,2139,612,2017,6,20,0,0,0,0,0,...,178,259,974,1143,832,1464,956,65,619,91


In [27]:
text_columns = [x for x in df.columns.tolist() if x.startswith("encoded")]
len(text_columns)

1344

In [35]:
text_df = df[text_columns]
text_df.head()

,encoded_1,encoded_2,encoded_3,encoded_4,encoded_5,encoded_6,encoded_7,encoded_8,encoded_9,encoded_10,...,encoded_1335,encoded_1336,encoded_1337,encoded_1338,encoded_1339,encoded_1340,encoded_1341,encoded_1342,encoded_1343,encoded_1344
0,0,0,0,0,0,0,0,0,0,0,...,523,170,1193,523,264,608,307,1143,794,1336
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,811
2,0,0,0,0,0,0,0,0,0,0,...,1106,44,921,633,776,902,794,239,824,997
3,0,0,0,0,0,0,0,0,0,0,...,878,178,394,1073,1258,1395,923,932,1015,285
4,0,0,0,0,0,0,0,0,0,0,...,178,259,974,1143,832,1464,956,65,619,91


In [47]:
MAX_SEQ_LEN

1344

In [54]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
    
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)

In [148]:
# build pytorch model
DROPOUT = 0.1

class Attention_Net(nn.Module):
    def __init__(self):
        super(Attention_Net, self).__init__()
        
        # define architecture
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBED_DIM)
#         self.embedding_dropout = nn.Dropout2d(DROPOUT) # take this out potentially
        
        self.lstm = nn.LSTM(EMBED_DIM, LSTM_DIM, bidirectional=True, batch_first=True) #batch_first=True?
        
        # attention layer
        self.attention_layer = Attention(LSTM_DIM * 2, MAX_SEQ_LEN) # param here could be wrong
        
        self.out = nn.Linear(LSTM_DIM * 2, 1)
        
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0)).view(1, MAX_SEQ_LEN, -1)
        h_lstm, _ = self.lstm(h_embedding)
        h_lstm_atten = self.attention_layer(h_lstm)
        out = self.out(h_lstm_atten)
        return out
            
        

In [149]:
train_X = torch.tensor(text_df.values)
train_Y = torch.tensor([[x] for x in df.helpful.values]).float()

print(train_X.size())
print(train_Y.size())

torch.Size([50, 1344])
torch.Size([50, 1])


In [36]:
MAX_SEQ_LEN = len(text_columns)
VOCAB_SIZE = 1492 # should ideally just transport this from prev
EMBED_DIM = 64
LSTM_DIM = 64

In [150]:
attention_model = Attention_Net()
loss_function = nn.MSELoss()
optimizer = optim.SGD(attention_model.parameters(), lr=0.001)


# out = attention_model(train_X)
# out.size()

In [151]:
# training loop
EPOCHS = 1

for i in range(EPOCHS):
    for idx, train_x_example in enumerate(train_X):
        attention_model.zero_grad()
        pred_y = attention_model(train_x_example)
        print ("pred_y = {}, actual = {}".format(pred_y, train_Y[idx]))
        loss = loss_function(pred_y, train_Y[idx])
        loss.backward()
        optimizer.step()
        print ("loss = ", loss)
        print ("\n")
        

pred_y = tensor([[0.1319]], grad_fn=<AddmmBackward>), actual = tensor([4.])
loss =  tensor(14.9626, grad_fn=<MseLossBackward>)


pred_y = tensor([[0.2321]], grad_fn=<AddmmBackward>), actual = tensor([0.])
loss =  tensor(0.0539, grad_fn=<MseLossBackward>)


pred_y = tensor([[0.2194]], grad_fn=<AddmmBackward>), actual = tensor([0.])
loss =  tensor(0.0481, grad_fn=<MseLossBackward>)


pred_y = tensor([[0.1167]], grad_fn=<AddmmBackward>), actual = tensor([1086.])
loss =  tensor(1179142.5000, grad_fn=<MseLossBackward>)


pred_y = tensor([[11.3361]], grad_fn=<AddmmBackward>), actual = tensor([2139.])
loss =  tensor(4526953.5000, grad_fn=<MseLossBackward>)


pred_y = tensor([[121.6173]], grad_fn=<AddmmBackward>), actual = tensor([55.])
loss =  tensor(4437.8647, grad_fn=<MseLossBackward>)


pred_y = tensor([[127.6925]], grad_fn=<AddmmBackward>), actual = tensor([228.])
loss =  tensor(10061.5908, grad_fn=<MseLossBackward>)


pred_y = tensor([[170.8452]], grad_fn=<AddmmBackward>), actual = tenso

In [153]:
# evaluate
with torch.no_grad():
    
    preds = attention_model(train_X[0])
    print ('predicted helpful = ', preds)
    print ('actual helpful = ', train_Y[0])

predicted helpful =  tensor([[0.6429]])
actual helpful =  tensor([4.])
